# Geoadmin migration path

## Current Issues
The current `mf-geoadmin3` app faces some issues that need to be addressed in the (near) future:
- end of life: angularJS (aka angular 1.X) is in LTS maintenance mode and will be discontinued in mid-2021
- slow: the current app is slow (see e.g. [performance analysis done for MVT](https://ltboc.infra.bgdi.ch/notebooks/mvt/ol-mapbox-performance-comparison.html) which shows that a considerable amount of time is lost during startup of the app)
- complex: despite the modularization, the current app is fairly complex (see dependency analysis below) and difficult to extend and maintain for non-angularJS experts

## Goals for next-gen app
The next-generation SPA should meet these goals:
- as much framework-agnostic code as possible: the new javascript standard ES6 introduced the concept of *modules*, allowing to natively modularize JS code without the need to use a framework (which was the case in the days when angularJS was invented)
- code documentation: the current code is hardly commented and documented, this should be improved
- follow the [Flux](https://facebook.github.io/flux/docs/in-depth-overview.html#content) principle: separate *state*, *logic* and *ui*
- modern build system: webpack is the de-facto standard (or at least very widely used) used to build (compile, minify) *module*-arized JS-libs
- local development: it should be possible to develop and serve the app locally (which comes for free when using webpack with webpack-dev-server)
- PWA: with choosing the correct JS frameworks, PWA's and/or native apps come almost for free (e.g. ionic, vue-native, ...)

## Development approach
In principle, there exist two fundamentally different ways of how to approach the development of a next-gen app, either a "green-field approach" or a "transformation approach", both with risks and advantages. The first question that needs to be answered for both of them is whether the feature set should remain exactly the same or whether some of the existing features can be discontinued. This is specially true for the numerous small exceptions here and there. The second question is whether development should happen in an agile manner controlled by us or as a task that is given to an external company and we'd get the final product in the end. In both cases we should take into account that we have no decision on projection system and rendering technology used in the future, hence we should design the new app in a way that allows to adapt to any imaginable scenario (e.g. by clearly separating stuff that is map-framework related).

### Green-field approach
A concise spec does not exist at the moment, however there's the existing code and a test-book that cover many of the use main cases (but leave out some of the aforementioned exceptions). A green-field approach would offer the possibility (if done in an agile way) to start with the most simple features (e.g. show one basemap) and gradually adding features existing in the current app, thereby deciding whether or not to keep this feature. The risks with the green-field approach are that 
- the necessary effort is hard to estimate (both in time and money)
- we would have to 'switch' at one point
- we would have to maintain the 'old' app while developing the new one
- we would potentially have to add features to the old app and the new one -> moving target
- if not done in a Agile way, we'd receive a black box in the end that would need a considerable effort again to understand
The advantages with the green-field approach are that
- we would end up with a clean app that is free from legacy code and logic
- code structure (components, class-APIs) would better mimick the current functionality

### Transformation approach
The biggest advantage of the transformation approach is that there's always a production-ready version of the app available. The transformation approach would implicitly require a careful analysis of the existing code, minimizing the risk of 'hidden' business logic being lost. Decisions on whether or not to keep and migrate features could be done incrementally. The disadvantage is that some extra work and code overhead is required due to the transformation. Furthermore, testing effort would increase during transformation phase, since it's hard and not feasible to cover all bridge code with automated tests. The risks with the transformation approach are that
- it would be more difficult to do major refactoring of the code structure, since migration would mostly happen component by component. Some refactoring could happen in a second step
- we would increase complexity during the transformation phase

The advantages would be that
- we could closely follow development and ensure/enforce code quality
- we would better understand the end product
- we would catch all edge cases and exceptions with the possibility to resolve them in a cleaner way (if possible)
- the discussion about features (and edge cases/exceptions) could happen step by step


# Project Mithridate
*Project Mithridate* is a proof of concept to demonstrate the feasibility of an 'in-app' migration path, gradually migrating parts of the code from the old to the new app. The basic idea is to use a wrapper (`ngVue` in the POC) to expose code in the 'new' app as angularJS modules that can be used in the old app.

## Dependency analysis
In order to find possible leaf nodes that could serve as a start for the migration and in general to understand the dependencies of the individual modules better, a dependency analysis has been made that is (partly) visualized in the following graph.
![geoadmin dependency graph](assets/graph.svg)


The code organisation looks as follows:
```mf-geoadmin:
src                                                 # the 'old' angularJS code
    |_components
    |_js
    |_lib
    |_...
vueApp
    |_dist                                          # contains the build libs, ready to be included in the 'old' angularjs app
        |_css
        |   |_appVueLib.css
        |_js
            |_appVueLib_NgVueBridge.js
            |_appVueLib_VendorsDependencies.js
            |_appVueLib.js
    |_src                                           # contains the 'new' app code
        |_DEV                                       # helper stuff needed only for development
        |   |_index.vue.js
        |_js                                        # contains framework-agnostic ES6 js-module code
        |   |_utils
        |       |_urlUtils.js
        |       |_...
        |_ngVueBridgeCode                           # contains code that only exists during migration phase
        |   |_ngVueComponentsModule.js
        |   |_ngVueDirectives.js
        |   |_ngVueServices.js
        |_vue                                       # contains the directly view-related components
            |_components
            |   |_ZoomButton.vue
            |   |_Translation
            |       |_LanguageSwitcher.vue
            |       |_LanguageSelector.vue
            |_App.vue                               # the entry-point of the 'new' app, once all functionality is migrated
    |_package.json                                  # new app has its own package.json
    |_webpack.config.js                             # new app is built using webpack (including local dev-server)
```

Code than can be framework agnostic should be framework agnostic and should go into `vueApp/src/js`. `vue` should only contain parts that are directly related to UI and contain as little business logic as possible.

Once the migration has finished and no code remains in the 'old' app, the `DEV` and `ngVueBridgeCode` can be removed as well as everything outside of `vueApp`.

## Working POC
A working POC (not perfect yet) can be inspected under https://mf-geoadmin3.dev.bgdi.ch/ltboc/src/index.html (https://github.com/geoadmin/mf-geoadmin3/tree/project_mithridate) where
- an existing service has been migrated to the new app (`gaGeomUtils`, now in `src/js/ol/geometryUtils.js` with bridge code in `src/ngVueBridgeCode/ngVueServices.js`)
- an existing directive with controller and service as well as a third party library has been migrated to vue (`$translate` and `gaLang`).

Note: handling translations in vue is better solved with using vue-i18n and not how it's done in the poc.

The transformation approach seems feasible so far, yet not everything has been done yet:
- make sure production build works
- check performance / increased size of build


## Todo
Things that remain to be done:

- decide on whether exact feature parity is required, think about new features that could be realized easier with the right code architecture 
- decide on which ui framework to use (strong favour for vue.js for its simplicity and size)
- decide on which test framework to use for new app

---
[1] https://exmachina.ch/technical/migrating-angular-to-vue/

[2] https://github.com/arcadeJHS/AngularVueIntegration

[3] https://github.com/ngVue/ngVue

[4] https://codewithhugo.com/from-angularjs-to-vue.js-commonjs-and-jest/

[5] https://github.com/google/closure-library/wiki/ES6-modules-and-Closure-Library

[6] https://stackoverflow.com/a/41537033 (using vuex store in external module)

[7] https://codeburst.io/vuex-getters-are-great-but-dont-overuse-them-9c946689b414 (best practice on vuex store)